<a href="https://colab.research.google.com/github/ShanmathiVasudevan/Siamese_NN_WiDS_SV/blob/main/week2/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!unzip -q -j /content/homer_bart.zip -d.

In [4]:
import csv,os
f=open('homer_bart.csv','w',newline='')
w=csv.writer(f,delimiter=',')
w.writerow(['File_name','Label'])
print(os.listdir('/content/'))
list_files=os.listdir('/content/')
list_imgfiles=[]
for file in list_files:
  if file[-4::]=='.bmp':
    list_imgfiles+=[file]
print(list_imgfiles)
for file in list_imgfiles:
  if file[0]=='b':
    w.writerow([file,0])
  else:
    w.writerow([file,1])

['.config', 'bart94.bmp', 'bart43.bmp', 'homer123.bmp', 'bart111.bmp', 'bart121.bmp', 'homer34.bmp', 'homer93.bmp', 'bart151.bmp', 'bart130.bmp', 'homer74.bmp', 'bart98.bmp', 'bart20.bmp', 'homer2.bmp', 'bart126.bmp', 'homer113.bmp', 'homer41.bmp', 'bart169.bmp', 'bart117.bmp', 'bart1.bmp', 'bart76.bmp', 'homer32.bmp', 'homer116.bmp', 'homer72.bmp', 'bart147.bmp', 'homer78.bmp', 'homer33.bmp', 'homer73.bmp', 'homer21.bmp', 'bart48.bmp', 'bart154.bmp', 'homer22.bmp', 'homer52.bmp', 'bart72.bmp', 'bart106.bmp', 'bart161.bmp', 'homer20.bmp', 'bart116.bmp', 'bart46.bmp', 'bart22.bmp', 'bart75.bmp', 'homer83.bmp', 'homer105.bmp', 'bart39.bmp', 'bart57.bmp', 'homer50.bmp', 'bart44.bmp', 'bart3.bmp', 'bart12.bmp', 'homer109.bmp', 'bart89.bmp', 'homer49.bmp', 'homer66.bmp', 'homer36.bmp', 'bart42.bmp', 'bart95.bmp', 'bart25.bmp', 'bart18.bmp', 'homer84.bmp', 'homer40.bmp', 'homer121.bmp', 'homer75.bmp', 'bart7.bmp', 'bart6.bmp', 'bart23.bmp', 'homer5.bmp', 'bart66.bmp', 'bart32.bmp', 'bart138.

In [5]:
import pandas as pd
data=pd.read_csv('homer_bart.csv')
print(data)

        File_name  Label
0      bart94.bmp      0
1      bart43.bmp      0
2    homer123.bmp      1
3     bart111.bmp      0
4     bart121.bmp      0
..            ...    ...
264    homer3.bmp      1
265    bart40.bmp      0
266     bart5.bmp      0
267    bart51.bmp      0
268    bart86.bmp      0

[269 rows x 2 columns]


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,regularizers
import numpy as np

In [7]:
filepaths=data['File_name'].values
labels=data['Label'].values
train=tf.data.Dataset.from_tensor_slices((filepaths[0:int(0.9*len(filepaths))],labels[0:int(0.9*len(labels))]))
test=tf.data.Dataset.from_tensor_slices((filepaths[int(0.9*len(filepaths)):],labels[int(0.9*len(labels)):]))


In [8]:
img_height=64
img_width=64
batch_size=32

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,random_brightness

zoom=tf.keras.layers.RandomZoom(0.2)
flip=tf.keras.layers.RandomFlip("horizontal")
translation=tf.keras.layers.RandomTranslation(height_factor=0.2,width_factor=0.2)
contrast=tf.keras.layers.RandomContrast(0.2)
def augment(image,label):
  image=flip(image)
  image=translation(image)
  #image=zoom(image)
  #image=contrast(image)
  return image,label




In [10]:
def read_image(image_file,label):
  image=tf.io.read_file("/content/"+image_file)
  image=tf.image.decode_bmp(image,channels=3)
  image=tf.image.resize(image,(img_height,img_width))
  return image,label

train=train.map(read_image).map(augment).batch(batch_size)
test=test.map(read_image).batch(batch_size)


In [13]:
inputs = keras.Input(shape=(img_height,img_width,3))
x=layers.Dense(192,activation='relu',kernel_regularizer=regularizers.l2(0.01))(inputs)
x=layers.BatchNormalization()(x)
x=layers.Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01))(x)
x=layers.BatchNormalization()(x)
'''x=layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01))(x)
x=layers.BatchNormalization()(x)'''
x=layers.Flatten()(x)
outputs=layers.Dense(2,activation='sigmoid',kernel_regularizer=regularizers.l2(0.02))(x)
model=keras.Model(inputs=inputs,outputs=outputs)
print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64, 64, 192)         │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 64, 64, 192)         │             768 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64, 64, 128)         │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 524288)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │       1,048,578 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,075,330 (4.10 MB)

 Trainable params: 1,074,690 (4.10 MB)

 Non-trainable params: 640 (2.50 KB)

None


In [14]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"],
)
model.fit(train,batch_size=batch_size,epochs=30,verbose=2)
model.evaluate(test,batch_size=batch_size,verbose=2)

Epoch 1/30
8/8 - 21s - 3s/step - accuracy: 0.6983 - loss: 3.1036
Epoch 2/30
8/8 - 17s - 2s/step - accuracy: 0.7231 - loss: 3.5043
Epoch 3/30
8/8 - 22s - 3s/step - accuracy: 0.7562 - loss: 3.4168
Epoch 4/30
8/8 - 19s - 2s/step - accuracy: 0.8099 - loss: 3.5481
Epoch 5/30
8/8 - 17s - 2s/step - accuracy: 0.8223 - loss: 2.6960
Epoch 6/30
8/8 - 21s - 3s/step - accuracy: 0.8760 - loss: 2.4311
Epoch 7/30
8/8 - 20s - 2s/step - accuracy: 0.8554 - loss: 2.9646
Epoch 8/30
8/8 - 19s - 2s/step - accuracy: 0.8554 - loss: 2.7704
Epoch 9/30
8/8 - 19s - 2s/step - accuracy: 0.8719 - loss: 2.5630
Epoch 10/30
8/8 - 17s - 2s/step - accuracy: 0.8306 - loss: 3.0153
Epoch 11/30
8/8 - 18s - 2s/step - accuracy: 0.8802 - loss: 2.7243
Epoch 12/30
8/8 - 31s - 4s/step - accuracy: 0.8884 - loss: 2.3197
Epoch 13/30
8/8 - 32s - 4s/step - accuracy: 0.8636 - loss: 2.9727
Epoch 14/30
8/8 - 19s - 2s/step - accuracy: 0.9298 - loss: 2.2446
Epoch 15/30
8/8 - 18s - 2s/step - accuracy: 0.8719 - loss: 2.3121
Epoch 16/30
8/8 - 2

[4.380598545074463, 0.8888888955116272]